# Final Tutorial: Analysis of Esports Earnings Across Different Games

Kevin Liang, Kevin Lopoi, Jordan Marry

## Introduction: eSports 

Over the last decade, the the gaming industry has become more and more popular. This large influx in popularity resulted in a boom in the competitve gaming scene, known as eSports. eSports is defined as a form of organized sport competition which involves video games, and is typically between professional players in an individual or a team setting.

While compeitions have always been popular in gaming culture, there wasn't particularly a "professional scene" for video games. Until the late 2000s, competitions have been mostly among amateurs. As more and more people began playing video games and video games become more popularized in our everyday life, viewers for tournaments and video game content in general began increasing as well. However, by the 2010s, eSports have become a "signficiant factor" in the video game industry as companies began to follow a "franchise approach" used in North American sports where there are financial sponsors looking to support teams.

Even to this day, eSports viewership has continued to increase as monthly eSports viewers have increased by 11.4% from 2020. The integration to popular culture and utilization of media outlets all contribute to this growth, and eSports organizations have produced many competitive organizers not only filled with players, but also influencers.
From 2017 to 2018, investments in the industry have skyrocketed from \\$490 million all the way to \\$4.5 billion. These investments are distributed across teams and players across many different games. Projections for this year has the industry on track to "surpass 1 billion dollars in revenue and hit 1.8 billion by 2022 according to Newzoo.
Read more about the growing revenue stream here: 
https://www.insiderintelligence.com/insights/esports-ecosystem-market-report/
This large amount of money does include sponsorships, advertisements, on top of tournament earnings itself. 

This dataset will look at tournament earnings alone, across 10 of the most popular games in the past decade:
Dota2, Counter-Strike: Global Offensive, Fortnite, League of Legends, Starcraft II, Overwatch, PUBG, Hearthstone, Heroes of the Storm, Arena of Valor

If you are curious and want to know more about the eSports industry, you can find out more at: 
https://en.wikipedia.org/wiki/Esports

## Purpose
This tutorial will involve utilizing techniques utilized in data science to analyze a data set containing the three CSV files for the "Highest Earning Players", "Highest Earning Teams", and a set of Countries and Continents to match the data with. 

The data will inlcude the

In [147]:
import pandas as pd
import re
import numpy as np
import requests

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup

In [ ]:
# Grabbing data from 2010 till now

years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]
dfs = []


session = requests.Session()
retry = Retry(connect=10, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Grabbing all the top 100 players from 2010-2021
for i in years:
    # players
    url = "https://www.esportsearnings.com/history/" + i + "/top_players"
    page = session.get(url)
    root = BeautifulSoup(page.content)
    root.prettify()
    table = root.find("table")
    
    rows = table.findAll('tr')
    headers = rows[0].findAll('td')
    headers = headers[1:]
    cols = []
    for head in headers:
        cols.append(head.text.strip())
    
    cols.insert(0, "Rank")
    cols.insert(1, "Year")
    cols.insert(2, "Country")
    cols[3] = "In-Game Name"
    cols.append("Game")
    cols.append("# of Tournaments")
    cols.remove("% of Total")
    
    players = rows[1:]
    
    rank = []
    country = []
    ign = []
    player_name = []
    total_year = []
    total_overall = []
    games = []
    tournaments = []
    
    for player in players:
        infos = player.findAll('td')
        rank.append(infos[0].text.strip())
        country.append(infos[1].find('img').get('title'))
        ign.append(infos[1].text.strip())
        player_name.append(infos[2].text.strip())
        total_year.append(infos[3].text.strip())
        total_overall.append(infos[4].text.strip())
        links = infos[1].findAll('a')
        url2 = "https://www.esportsearnings.com/" + links[1].get('href')
        page = session.get(url2)
        root = BeautifulSoup(page.content)
        title = root.find("h1")
        title = title.text.strip()
        games.append(title)
        
        #tourney = root.findAll('div', {'class' : 'format_cell info_text_value'})
        
        tourney = root.find_all(text=re.compile(' From ' + r'[0-9]+' + ' Tournament'))
        stri = tourney[0].split()
        tournaments.append(stri[1])
        
        
        
    
    df = pd.DataFrame(columns = cols)
    df['Rank'] = rank
    df['Year'] = i
    df['Country'] = country
    df['In-Game Name'] = ign
    df['Player Name'] = player_name
    df['Total (Year)'] = total_year
    df['Total (Overall)'] = total_overall
    df['Game'] = games
    df['# of Tournaments'] = tournaments
    
    dfs.append(df)
    


In [ ]:
df = pd.concat(dfs)
df.to_csv('players.csv')

In [ ]:
df = pd.read_csv('players.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
games = df['Game'].str.split("- ")
df['Game'] = games

In [ ]:
df = df.astype({"Rank": int})

In [ ]:
for i in range(0,len(df)):
    df.at[i,'Game'] = df.loc[:, "Game"][i][-1]

In [ ]:
df['Game'] = df['Game'].str.replace(" Player","")
df